![](http://)* Importando as bibliotecas


In [ ]:
import warnings
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

* Importação dos dados

In [ ]:
cogumelos = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

* Visualização prévia dos dados gerais

In [ ]:
cogumelos.head()

* Vendo se existe algum valor nulo

In [ ]:
cogumelos.isnull().sum()

Descrição da tabela

In [ ]:
cogumelos.describe()

* Separando a coluna de resposta do resto da tabela e apagando a coluna veil-type pois os valores do dataset são todos iguais, se tornando irrelevante para o trabalho

In [ ]:
y=cogumelos['class']
X=cogumelos.drop('class',axis=1)
X=cogumelos.drop('veil-type',axis=1)
X.head()

* Transformando valores não numéricos em numéricos

In [ ]:
from sklearn.preprocessing import LabelEncoder
Encoder_X = LabelEncoder() 
for col in X.columns:
    X[col] = Encoder_X.fit_transform(X[col])
Encoder_y=LabelEncoder()
y = Encoder_y.fit_transform(y)

* Mostrando tabela X modificada

In [ ]:
X.head()

* Mostrando array y que contém as respostas

In [ ]:
print (y)

* Convertendo valores categóricos em indicadores 

In [ ]:
X=pd.get_dummies(X,columns=X.columns,drop_first=True)
X.head()

* Criando uma variiável nova para fazer as plotagens e convertendo letras em numeros, para poder criar as tabelas

In [ ]:
cogumelos_View = cogumelos;
labelencoder=LabelEncoder()
for column in cogumelos_View.columns:
    cogumelos_View[column] = labelencoder.fit_transform(cogumelos_View[column]);

In [ ]:
cogumelos_View=cogumelos_View.drop(["veil-type"],axis=1)

* Mostrando a distribuição dos cogumelos venenosos de acordo com as caracteristicas do dataset

In [ ]:
cogumelos_div = pd.melt(cogumelos_View, "class", var_name="Characteristics")
fig, ax = plt.subplots(figsize=(10,5))
p = sns.violinplot(ax = ax, x="Characteristics", y="value", hue="class", split = True, data=cogumelos_div, inner = 'quartile', palette = 'Set2')
cogumelos_no_class = cogumelos_View.drop(["class"],axis = 1)
p.set_xticklabels(rotation = 90, labels = list(cogumelos_no_class.columns));

* Mostrando a quantidade de comestiveis e venenosos

In [ ]:
plt.figure()
pd.Series(cogumelos_View['class']).value_counts().sort_index().plot(kind = 'bar')
plt.ylabel("Quantidade")
plt.xlabel("Classe")
plt.title('Quantidade de comestiveis e venenosos (0=Comestiveis, 1=Venenosos)');

* Vendo a correlação entre as variáveis

In [ ]:
plt.figure(figsize=(35,16))
sns.heatmap(cogumelos_View.corr(),linewidths=.1,cmap="YlGnBu", annot=True)
plt.yticks(rotation=0);

* Dividindo os dados em teste e treino

In [ ]:
from sklearn.model_selection import train_test_split
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.4, random_state=42)

* Escalonando os dados

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_treino = sc.fit_transform(X_treino)
X_teste = sc.transform(X_teste)

* Reduzindo a dimensionalidade linear com o PCA para projedar os dados em um espaco dimensional inferior

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

X_treino = pca.fit_transform(X_treino)
X_teste = pca.transform(X_teste)

* Função para vizualizar os dados de treino e teste

In [ ]:
def visualizar_treino(modelo):
    sns.set_context(context='notebook',font_scale=2)
    plt.figure(figsize=(16,9))
    from matplotlib.colors import ListedColormap
    X_set, y_set = X_treino, y_treino
    X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
    plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.6, cmap = ListedColormap(('#ff0000', '#33cc33')))
    plt.xlim(X1.min(), X1.max())
    plt.ylim(X2.min(), X2.max())
    for i, j in enumerate(np.unique(y_set)):
        plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                    c = ListedColormap(('#cc0000', '#248f24'))(i), label = j)
    plt.title("%s Treino" %(modelo))
    plt.legend();
    
def visualizar_teste(modelo):
    sns.set_context(context='notebook',font_scale=2)
    plt.figure(figsize=(16,9))
    from matplotlib.colors import ListedColormap
    X_set, y_set = X_teste, y_teste
    X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                         np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
    plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
                 alpha = 0.6, cmap = ListedColormap(('#ff0000', '#33cc33')))
    plt.xlim(X1.min(), X1.max())
    plt.ylim(X2.min(), X2.max())
    for i, j in enumerate(np.unique(y_set)):
        plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                    c = ListedColormap(('#cc0000', '#248f24'))(i), label = j)
    plt.title("%s Teste " %(modelo))
    plt.legend();

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag, name):
    if flag == True:
        print("Resultado do treino utilizando " + name + ":\n")
        print('Precisão do resultado: {0:.4f}\n'.format(accuracy_score(y_treino,classifier.predict(X_treino))))
    elif flag == False:
        print("Resultado do teste ultizando " + name + ":\n")
        print('Precisão do resultado: {0:.4f}\n'.format(accuracy_score(y_teste,classifier.predict(X_teste))))

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

classifier.fit(X_treino,y_treino)

In [ ]:
visualizar_treino('Regressão Logística')

In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=True,name="Regressão Logística")

In [ ]:
visualizar_teste('Regressão Logística')

In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=False,name="Regressão Logística")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN

classifier = KNN()
classifier.fit(X_treino,y_treino)

In [ ]:
visualizar_treino('K-Nearest Neighbors')


In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=True,name="K-Nearest Neighbors")

In [ ]:
visualizar_teste('K-Nearest Neighbors')


In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=False,name="K-Nearest Neighbors")


In [ ]:
from sklearn.tree import DecisionTreeClassifier as DT

classifier = DT(criterion='entropy',random_state=42)
classifier.fit(X_treino,y_treino)

In [ ]:
visualizar_treino('Árvore de Decisão')

In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=True, name="Árvore de Decisão")

In [ ]:
visualizar_teste('Árvore de Decisão')

In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=False, name="Árvore de Decisão ")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42)
classifier.fit(X_treino, y_treino)

In [ ]:
visualizar_treino('Random Forest')

In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=True, name="Random Forest")

In [ ]:
visualizar_teste('Random Forest')

In [ ]:
printando_resultado(classifier,X_treino,y_treino,X_teste,y_teste,flag=False, name="Random Forest")